In [ ]:
#!/usr/bin/env python3

import sys
import os
import numpy as np
import subprocess
import pandas as pd
import pyperclip
import math
import matplotlib.pyplot as plt

def build_graph (X:np.array, Y:np.array, x_name:str='', y_name:str='', x_dim:str = '', y_dim:str = '', name:str = '', no_line:bool = True, color:str = 'black', color_dot:str = 'blue', include_zero:bool = False, dx:np.array = np.array ([]), dy:np.array = np.array ([]), function:str='none', aprx_l=0, aprx_r=-1, figure=0, line_name='', aprx_only=False):

    k = -1.0
    b = -1.0

    xlimL = min (X) * 0.8
    ylimL = min (Y) * 0.8
    xlimR = max (X) * 1.2
    ylimR = max (Y) * 1.2
    if (include_zero):
        xlimL = min (0, xlimL)
        ylimL = min (0, ylimL)
        xlimR = max (0, xlimR)
        ylimR = max (0, ylimR)

    ax1 = 0
    if (figure == 0):
        fig = plt.figure (figsize=(16, 9), dpi=200)
        ax1 = fig.add_subplot (111)
        ax1.set_title (rf'{name}', fontsize=17)
    else:
        fig = figure
        ax1 = fig.axes[0]
        xlimL = min (ax1.get_xlim()[0], xlimL)
        xlimR = max (ax1.get_xlim()[1], xlimR)
        ylimL = min (ax1.get_ylim()[0], ylimL)
        ylimR = max (ax1.get_ylim()[1], ylimR)


    if (no_line == False): ax1.plot (X, Y, color = color, linestyle = '-', markersize=0, linewidth=0.5)
    if (aprx_only == False):
        if (line_name == 0):ax1.scatter (X, Y, label = rf'{y_name}({x_name})', color = color_dot, marker='+', s=50, linewidths=1)
        else: ax1.scatter (X, Y, label = rf'{y_name}({x_name})' + '   : ' + str(line_name), color = color_dot, marker='+', s=50, linewidths=1)

    if function == 'lin':
        if (aprx_r == -1):
            aprx_r = len (X)
        model = np.poly1d (np.polyfit (X[aprx_l:aprx_r], Y[aprx_l:aprx_r], deg = 1))
        approx = rf'$({str (model[1])}) * x + ({str (model[0])})$' + '   : ' + str (line_name)
        k = float (model[1])
        b = float (model[0])
        x_range = np.linspace (min(X) * 0.9, max (X) * 1.1, 2)
        ax1.plot (x_range, model (x_range), '--', color = 'orange', label = approx)

    ax1.set_xlabel (rf'{x_name}, {x_dim}', size=20)
    ax1.set_ylabel (rf'{y_name}, {y_dim}', size=20)
    ax1.set_xlim (xlimL, xlimR)
    ax1.set_ylim (ylimL, ylimR)
    ax1.legend (fontsize=10)
    ax1.grid ()
    return (fig, (k, b))



#write_as_column - outputs as table column
#vertical=True - outputs a transponded version
#vertical=False - outputs a non transponded version
#if total_rows > amount of resulting rows then add empty elements
#if total_columns > amount of resulting columns then add empty columns
def arr_to_tex (arr:np.ndarray, outfile:str = '/dev/null', append:bool = False, to_clip:bool = False, precision=2, as_column:bool=False, vertical:bool=False, header:bool=True, col_total:int=0, row_total:int=0, caption:str = ''):
    tex_out =''
    if (header):
        tex_out += '\n\\begin{table}[H]\n\\caption{' + caption + '}\n\\begin{center}\n\\begin{tabular}{|c|*{15}{c|}}\\hline\n'

    if (arr.ndim == 1):
        for i in arr:
            if (isinstance (i, str)): tex_out += i
            else: tex_out += str (round (i, precision))
            tex_out += ' & '
            if (as_column): tex_out += '\n'
        if (as_column): tex_out += ' & \n' * max (0, row_total - len (arr))
        else: tex_out = tex_out[:-3] +  ' & ' * max (0, col_total - len (arr)) + '\\\\\\hline\n'
    elif (arr.ndim == 2):
        if (vertical):
            row_n = arr.shape[0]
            col_n = arr.shape[1]
            for i in range (row_n):
                for j in range (col_n):
                    if (isinstance (arr[j][i], str)): tex_out += arr[j][i]
                    else: tex_out += str (round (arr[j][i], precision))
                    tex_out += ' & '
                tex_out = tex_out[:-3] +  ' & ' * max (0, col_total - col_n) + '\\\\\\hline\n'
            tex_out += (' & ' * col_total + '\\\\\\hline\n') * row_total
        else:
            row_n = arr.shape[1]
            col_n = arr.shape[0]
            for i in arr:
                for j in i:
                    if (isinstance (j, str)): tex_out += j
                    else: tex_out += str (round (j, precision))
                    tex_out += ' & '
                tex_out = tex_out[:-3] +  ' & ' * max (0, col_total - col_n) + '\\\\\\hline\n'
            tex_out += (' & ' * col_total + '\\\\\\hline\n') * row_total

    if (header): tex_out += '\\end{tabular}\n\\end{center}\n\\end{table}\n\n'

    if (to_clip):
        clip_content = ''
        if (append): clip_content = pyperclip.paste ()
        clip_content += tex_out
        pyperclip.copy (clip_content)

    open_str = 'w'
    if (append): open_str = 'a'

    with open (outfile, open_str) as output:
        print (tex_out, file = output)
